<a href="https://colab.research.google.com/github/syedsajidhussain/3D-GAN/blob/main/Ph%C3%A2n_lo%E1%BA%A1i_%E1%BA%A3nh_si%C3%AAu_%C3%A2m_tim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import cv2
import numpy as np
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report

In [2]:
!git clone https://github.com/ngohonggiang/Echocardiography.git

Cloning into 'Echocardiography'...
remote: Enumerating objects: 8338, done.
remote: Total 8338 (delta 0), reused 0 (delta 0), pack-reused 8338
Receiving objects: 100% (8338/8338), 488.02 MiB | 27.90 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Checking out files: 100% (8328/8328), done.


In [3]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_data():
  size = 224
  # raw
  transform_train_raw = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor()
  ])

  # preprocess
  transform_train_preprocess = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.GaussianBlur(3),
    transforms.RandomRotation(2.8),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.ToTensor()
  ])

  # augmentation
  transform_train_augmentation = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.RandomCrop(size, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.1),
    transforms.ToTensor()
  ])

  # test
  transform_test = transforms.Compose([
    transforms.Resize((size, size)),                                 
    transforms.ToTensor()
  ])

  trainset = torchvision.datasets.ImageFolder(
      root='/content/Echocardiography/DATA_CHAMBER_2021/train', transform=transform_train_augmentation)
  testset = torchvision.datasets.ImageFolder(
      root='/content/Echocardiography/DATA_CHAMBER_2021/test', transform=transform_test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  batch_size = 32
  trainloader = DataLoader(dataset=datasets.train, batch_size=batch_size, shuffle=True, num_workers=2)
  testloader = DataLoader(dataset=datasets.test, batch_size=batch_size, shuffle=False, num_workers=2)
  return TrainTest(train=trainloader, test=testloader)


In [4]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_step = 40
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    
    outputs = model(images)
    loss = loss_func(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    running_loss += loss.item()
    if i % reporting_step == reporting_step - 1:
        print(f"Epoch {epoch} Step {i} ave_loss {running_loss/reporting_step:0.4f}")
        running_loss = 0.0

def test_epoch(epoch, model, loader, device):
  pred = []
  true = []
  with torch.no_grad():
    model.eval()
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      pred += list(predicted.cpu().numpy())
      true += list(labels.cpu().numpy())
  return pred, true

In [5]:
def main(model=None):
  classes = get_classes()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print(device)

  if model == 'vgg16':
    PATH = './vgg16.pth'
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.Linear(in_features=4096, out_features=3)
  elif model == 'vgg19':
    PATH = './vgg19.pth'
    model = torchvision.models.vgg19()
    model.classifier[6] = torch.nn.modules.Linear(in_features=4096, out_features=3)
  elif model == 'resnet50':
    PATH = './resnet50.pth'
    model = torchvision.models.resnet50()
    model.fc = torch.nn.modules.Linear(in_features=2048, out_features=3)

  model.to(device)
  loss_function = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_function, optimizer, device)
    pred, true = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(true, pred, target_names=classes))  
    print("---------------------------------------------------------------")      

    torch.save(model.state_dict(), PATH)
  return model

In [6]:
modelVGG16 = main(model='vgg16')

cuda:0
Epoch 0 Step 39 ave_loss 1.0958
Epoch 0 Step 79 ave_loss 1.0712
Epoch 0 Step 119 ave_loss 1.0755
Epoch 0 Step 159 ave_loss 1.0959
Epoch 0 Step 199 ave_loss 1.0925
              precision    recall  f1-score   support

          2C       0.31      1.00      0.48       409
          3C       0.00      0.00      0.00       367
          4C       1.00      0.10      0.19       831

    accuracy                           0.31      1607
   macro avg       0.44      0.37      0.22      1607
weighted avg       0.60      0.31      0.22      1607

---------------------------------------------------------------
Epoch 1 Step 39 ave_loss 1.0670
Epoch 1 Step 79 ave_loss 1.0429
Epoch 1 Step 119 ave_loss 0.8980
Epoch 1 Step 159 ave_loss 0.8524
Epoch 1 Step 199 ave_loss 0.6616
              precision    recall  f1-score   support

          2C       0.78      0.45      0.57       409
          3C       0.49      0.68      0.57       367
          4C       0.83      0.87      0.85       831

    

In [7]:
modelVGG19 = main(model='vgg19')

cuda:0
Epoch 0 Step 39 ave_loss 1.0964
Epoch 0 Step 79 ave_loss 1.0915
Epoch 0 Step 119 ave_loss 1.0927
Epoch 0 Step 159 ave_loss 1.0541
Epoch 0 Step 199 ave_loss 1.0076
              precision    recall  f1-score   support

          2C       0.47      0.70      0.56       409
          3C       0.33      0.72      0.45       367
          4C       1.00      0.23      0.38       831

    accuracy                           0.46      1607
   macro avg       0.60      0.55      0.46      1607
weighted avg       0.71      0.46      0.44      1607

---------------------------------------------------------------
Epoch 1 Step 39 ave_loss 0.9075
Epoch 1 Step 79 ave_loss 0.8735
Epoch 1 Step 119 ave_loss 0.7328
Epoch 1 Step 159 ave_loss 0.7013
Epoch 1 Step 199 ave_loss 0.5160
              precision    recall  f1-score   support

          2C       0.69      0.72      0.70       409
          3C       0.63      0.44      0.52       367
          4C       0.84      0.93      0.88       831

    

In [8]:
modelResNet50 = main(model='resnet50')

cuda:0
Epoch 0 Step 39 ave_loss 3.3247
Epoch 0 Step 79 ave_loss 1.3071
Epoch 0 Step 119 ave_loss 1.1558
Epoch 0 Step 159 ave_loss 1.3079
Epoch 0 Step 199 ave_loss 1.1154


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          2C       0.39      0.78      0.52       409
          3C       0.30      0.64      0.40       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.34      1607
   macro avg       0.23      0.47      0.31      1607
weighted avg       0.17      0.34      0.22      1607

---------------------------------------------------------------
Epoch 1 Step 39 ave_loss 1.0985
Epoch 1 Step 79 ave_loss 1.1791
Epoch 1 Step 119 ave_loss 0.9669
Epoch 1 Step 159 ave_loss 0.8986
Epoch 1 Step 199 ave_loss 0.8078
              precision    recall  f1-score   support

          2C       0.58      0.83      0.68       409
          3C       0.98      0.15      0.26       367
          4C       0.73      0.84      0.78       831

    accuracy                           0.68      1607
   macro avg       0.76      0.61      0.57      1607
weighted avg       0.75      0.68      0.63      1607

-----------